In [32]:
import pandas as pd
import numpy as np
import requests

In [3]:
barts_hotspots = pd.read_csv('data/barts_hotspots.csv')
barts_to_all = pd.read_csv('data/barts_to_all.csv')
hotspots_to_all = pd.read_csv('data/hotspots_to_all.csv')
hours_q1 = pd.read_csv('data/hours_q1.csv')
hours_q2 = pd.read_csv('data/hours_q2.csv')

In [146]:
barts_hotspots_filled.to_csv('barts_hotspots_updated.csv')

In [50]:
barts_hotspots.columns = ['Date',
 'Origin ID',
 'Origin Name',
 'Destination ID',
 'Destination Name',
 'Daily Mean Travel Time',
 'Daily Range - Lower',
 'Daily Range - Upper',
 'AM Mean Travel Time',
 'AM Range - Lower',
 'AM Range - Upper',
 'PM Mean Travel Time',
 'PM Range - Lower Bound Travel Time',
 'PM Range - Upper Bound Travel Time',
 'Midday Mean Travel Time',
 'Midday Range - Lower',
 'Midday Range - Upper',
 'Evening Mean Travel Time',
 'Evening Range - Lower',
 'Evening Range - Upper',
 'Early Morning Mean Travel Time',
 'Early Morning Range - Lower',
 'Early Morning Range - Upper',
 'weekday']

In [52]:
barts_hotspots.sort_values('Date')

,Date,Origin ID,Origin Name,Destination ID,Destination Name,Daily Mean Travel Time,Daily Range - Lower,Daily Range - Upper,AM Mean Travel Time,AM Range - Lower,...,Midday Mean Travel Time,Midday Range - Lower,Midday Range - Upper,Evening Mean Travel Time,Evening Range - Lower,Evening Range - Upper,Early Morning Mean Travel Time,Early Morning Range - Lower,Early Morning Range - Upper,weekday
2106,2019-01-01,3792,"Oracle Park, 24 Willie Mays Plaza, San Francis...",3603,"Embarcadero, San Francisco, CA",684.0,381.0,1228.0,NaN,NaN,...,432.0,338.0,551.0,NaN,NaN,NaN,851.0,478.0,1515.0,1
2524,2019-01-01,3792,"Oracle Park, 24 Willie Mays Plaza, San Francis...",3692,2nd Street and Stevenson Street (Montgomery BA...,483.0,319.0,729.0,NaN,NaN,...,464.0,343.0,627.0,427.0,322.0,565.0,493.0,312.0,777.0,1
2091,2019-01-01,3603,"Embarcadero, San Francisco, CA",3396,"The Palace Of Fine Arts, 3601 Lyon St, San Fra...",1315.0,1074.0,1609.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2891,2019-01-01,3792,"Oracle Park, 24 Willie Mays Plaza, San Francis...",3760,"Powell BART Station, Market St and Powell St, ...",677.0,453.0,1012.0,NaN,NaN,...,572.0,426.0,769.0,NaN,NaN,NaN,730.0,467.0,1139.0,1
2642,2019-01-01,3396,"The Palace Of Fine Arts, 3601 Lyon St, San Fra...",3692,2nd Street and Stevenson Street (Montgomery BA...,1559.0,1233.0,1970.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1809.0,1551.0,2110.0,1
3005,2019-01-01,3396,"The Palace Of Fine Arts, 3601 Lyon St, San Fra...",3760,"Powell BART Station, Market St and Powell St, ...",1576.0,1279.0,1942.0,NaN,NaN,...,1545.0,1248.0,1912.0,NaN,NaN,NaN,1685.0,1427.0,1989.0,1
2531,2019-01-01,3692,2nd Street and Stevenson Street (Montgomery BA...,3792,"Oracle Park, 24 Willie Mays Plaza, San Francis...",517.0,274.0,973.0,NaN,NaN,...,396.0,248.0,630.0,311.0,184.0,525.0,571.0,297.0,1096.0,1
2672,2019-01-01,3394,"Fisherman's Wharf, San Francisco, CA",3692,2nd Street and Stevenson Street (Montgomery BA...,1125.0,756.0,1674.0,NaN,NaN,...,943.0,717.0,1240.0,761.0,631.0,918.0,1736.0,1011.0,2983.0,1
2078,2019-01-01,3394,"Fisherman's Wharf, San Francisco, CA",3603,"Embarcadero, San Francisco, CA",1124.0,670.0,1886.0,NaN,NaN,...,869.0,618.0,1221.0,643.0,508.0,814.0,1516.0,851.0,2699.0,1
1864,2019-01-01,3396,"The Palace Of Fine Arts, 3601 Lyon St, San Fra...",3603,"Embarcadero, San Francisco, CA",1450.0,1115.0,1886.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [64]:
def stats(df):
    stats = df.describe().T
    stats['filled'] = stats['count']/len(df)
    return stats[['filled']]

In [147]:
#barts_hotspots_filled = barts_hotspots.copy()
barts_hotspots_filled['Destination Name'] = barts_hotspots_filled['Destination Name'].apply(lambda x: "Fisherman's Wharf, 286-298 Jefferson St, San Francisco, CA" if "Fisherman" in x else x)
barts_hotspots_filled['Destination Name'].nunique()

6

In [85]:
def conditional_fill(df,col):
    df_filled = df.copy()
    if 'list' in str(type(col)):
        for i in col:
            df_filled[i] = barts_hotspots_filled.groupby(['Origin ID','weekday','Destination ID'])[i].transform(lambda x: x.fillna(x.median()))
    return df_filled

In [93]:
col_to_fill = list(barts_hotspots_filled.columns)
col_to_fill.remove('Date')
col_to_fill.remove('Origin ID')
col_to_fill.remove('Origin Name')
col_to_fill.remove('Destination ID')
col_to_fill.remove('Destination Name')
col_to_fill

['Daily Mean Travel Time',
 'Daily Range - Lower',
 'Daily Range - Upper',
 'AM Mean Travel Time',
 'AM Range - Lower',
 'AM Range - Upper',
 'PM Mean Travel Time',
 'PM Range - Lower Bound Travel Time',
 'PM Range - Upper Bound Travel Time',
 'Midday Mean Travel Time',
 'Midday Range - Lower',
 'Midday Range - Upper',
 'Evening Mean Travel Time',
 'Evening Range - Lower',
 'Evening Range - Upper',
 'Early Morning Mean Travel Time',
 'Early Morning Range - Lower',
 'Early Morning Range - Upper',
 'weekday']

In [94]:
barts_hotspots_filled = conditional_fill(barts_hotspots_filled,col_to_fill)

In [108]:
stats(barts_hotspots_filled)

,filled
Origin ID,1.000000
Destination ID,1.000000
Daily Mean Travel Time,1.000000
Daily Range - Lower,1.000000
Daily Range - Upper,1.000000
AM Mean Travel Time,0.857274
AM Range - Lower,0.857274
AM Range - Upper,0.857274
PM Mean Travel Time,0.960098
PM Range - Lower Bound Travel Time,0.960098


In [96]:
stats(barts_hotspots)

,filled
Origin ID,1.000000
Destination ID,1.000000
Daily Mean Travel Time,0.982198
Daily Range - Lower,0.982198
Daily Range - Upper,0.982198
AM Mean Travel Time,0.550031
AM Range - Lower,0.550031
AM Range - Upper,0.550031
PM Mean Travel Time,0.715163
PM Range - Lower Bound Travel Time,0.715163


In [109]:
barts_hotspots_filled.head()

,Date,Origin ID,Origin Name,Destination ID,Destination Name,Daily Mean Travel Time,Daily Range - Lower,Daily Range - Upper,AM Mean Travel Time,AM Range - Lower,...,Midday Mean Travel Time,Midday Range - Lower,Midday Range - Upper,Evening Mean Travel Time,Evening Range - Lower,Evening Range - Upper,Early Morning Mean Travel Time,Early Morning Range - Lower,Early Morning Range - Upper,weekday
0,2019-06-09,3396,"The Palace Of Fine Arts, 3601 Lyon St, San Fra...",3603,"Embarcadero, San Francisco, CA",1588.0,1054.0,2392.0,NaN,NaN,...,1398.5,1191.5,1640.5,1149.5,861.0,1535.0,NaN,NaN,NaN,6
1,2019-04-28,3603,"Embarcadero, San Francisco, CA",3394,"Fisherman's Wharf, San Francisco, CA",639.0,512.0,796.0,579.0,427.0,...,654.0,524.0,817.0,608.0,492.0,753.0,NaN,NaN,NaN,6
2,2019-05-18,3603,"Embarcadero, San Francisco, CA",3394,"Fisherman's Wharf, San Francisco, CA",730.0,520.0,1024.0,567.0,460.0,...,791.0,590.0,1061.0,569.0,406.0,797.0,NaN,NaN,NaN,5
3,2019-04-18,3792,"Oracle Park, 24 Willie Mays Plaza, San Francis...",3603,"Embarcadero, San Francisco, CA",512.0,371.0,705.0,648.0,461.0,...,528.0,398.0,702.0,442.0,321.0,608.0,363.0,297.0,443.0,3
4,2019-04-08,3792,"Oracle Park, 24 Willie Mays Plaza, San Francis...",3603,"Embarcadero, San Francisco, CA",505.0,364.0,701.0,621.0,489.0,...,429.0,339.0,542.0,502.0,354.0,711.0,377.0,311.0,467.0,0


In [135]:
test_barts_hotspots_filled = barts_hotspots_filled[['Destination Name',"Origin Name",'Date','weekday','AM Mean Travel Time','PM Mean Travel Time']]

In [136]:
num = 20
test_barts_hotspots_filled_melt = pd.melt(test_barts_hotspots_filled, id_vars=['Destination Name',"Origin Name", 'Date','weekday'], 
        var_name="AM or PM", 
        value_name="Value")
test_barts_hotspots_filled_melt_sample = test_barts_hotspots_filled_melt.sample(num)
test_barts_hotspots_filled_melt_sample.head(num)

,Destination Name,Origin Name,Date,weekday,AM or PM,Value
1715,"The Palace Of Fine Arts, 3601 Lyon St, San Fra...","Embarcadero, San Francisco, CA",2019-03-31,6,AM Mean Travel Time,NaN
5878,2nd Street and Stevenson Street (Montgomery BA...,"Oracle Park, 24 Willie Mays Plaza, San Francis...",2019-01-18,4,PM Mean Travel Time,640.0
2033,"Embarcadero, San Francisco, CA","Oracle Park, 24 Willie Mays Plaza, San Francis...",2019-02-02,5,AM Mean Travel Time,378.0
5926,"The Palace Of Fine Arts, 3601 Lyon St, San Fra...","2nd Street and Stevenson Street, San Francisco...",2019-03-08,4,PM Mean Travel Time,1364.0
3986,2nd Street and Stevenson Street (Montgomery BA...,"Fisherman's Wharf, San Francisco, CA",2019-06-13,3,PM Mean Travel Time,2135.0
1740,"The Palace Of Fine Arts, 3601 Lyon St, San Fra...","Embarcadero, San Francisco, CA",2019-01-06,6,AM Mean Travel Time,NaN
3145,"Fisherman's Wharf, San Francisco, CA","Powell BART Station, Market St and Powell St, ...",2019-01-09,2,AM Mean Travel Time,916.0
1991,"Oracle Park, 24 Willie Mays Plaza, San Francis...","Embarcadero, San Francisco, CA",2019-01-14,0,AM Mean Travel Time,687.0
68,"Embarcadero, San Francisco, CA","Fisherman's Wharf, San Francisco, CA",2019-04-19,4,AM Mean Travel Time,832.0
22,"Fisherman's Wharf, San Francisco, CA","Embarcadero, San Francisco, CA",2019-05-11,5,AM Mean Travel Time,601.0


In [ ]:
def recommend(df, origin, destination, *argv):
    copy = df.copy()
    output = copy[(copy['Origin Name']==origin)&(copy['Destination Name']==destination)]
    output.groupby()

        